In [6]:
import os 
import sys
import bbf_twitter_secrets2 #remove keys
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import twitter
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from textblob import TextBlob

import sqlite3
import time
from datetime import datetime
import re

In [7]:
conn = sqlite3.connect('twitter.db')
c = conn.cursor()

In [ ]:
# Run this only the first time you use it so you don't keep re-creating tables
c.execute('''CREATE TABLE twitter
             (filter text, date text, username text, tweet text)''')

In [8]:
consumer_key = bbf_twitter_secrets2.consumerKey1()
consumer_secret = bbf_twitter_secrets2.consumerSecret1()
access_token = bbf_twitter_secrets2.accessToken1()
access_token_secret = bbf_twitter_secrets2.accessSecret1()

In [9]:
filter_input = 'Last Jedi'
class listener(StreamListener):
    def __init__(self, time_limit=60): # how many seconds you want it to run for
        self.start_time = time.time()
        self.limit = time_limit
        super(listener, self).__init__()
        
    def on_data(self, data):
        if (time.time() - self.start_time) < self.limit:
            all_data = json.loads(data)
            tweet = all_data["text"]
            username = all_data["user"]["screen_name"]        
            c.execute("INSERT INTO twitter (filter, date, username, tweet) VALUES (?,?,?,?);", 
                      (filter_input, datetime.now(), username, tweet))
            conn.commit()
            print(username, tweet) 
            row_num = c.execute("SELECT COUNT (*) FROM twitter")
            row_num1 = row_num.fetchall()
            print(row_num1)
            return True
        else:
            return False
        
    def on_error(self, status):
        print(status)
        
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitterStream = Stream(auth, listener())
twitterStream.filter(track=[filter_input], languages=["en"])

zaric RT @EmperorBigD: You know its funny.

I haven't seen a SINGLE person complain that they hated Last Jedi because "THEIR theory" didn't happe…
[(53,)]
Tutvm Also I finally got to see the last Jedi today. 
Strong overall but some weird moments &amp; a terrible (though maybe ne… https://t.co/1dtfXmnjtG
[(54,)]
TrisYoung28 Watching The Last Jedi again by myself on Christmas day sounds great
[(55,)]
TheFearlessHero RT @z0mgItsHutch: "Jesus Christ I can't believe you'd rate The Last Jedi so high. Seriously what is wrong with you??"

Good question, and t…
[(56,)]
Devidraws The last Jedi was fucking boring as hell
[(57,)]
EmmaLindhagen Without having seen the last Jedi I can honestly say:

Mood https://t.co/Dlm1r0SBeZ
[(58,)]
NihonbashiB Check out Figure King 239 Japanese Magazine Star Wars The Last Jedi HOT TOYS   https://t.co/wupibT5A5w via @eBay
[(59,)]
vasira96 oh boy the Hot fandom discourse over the last jedi yIKES maybe take a break a get a drink and realize that just lik… https://

In [10]:
cur = c.execute('SELECT * FROM twitter WHERE filter=?', (filter_input,)) 
look = cur.fetchall()

In [11]:
look

[('Last Jedi',
  '2017-12-22 17:34:21.800355',
  'zaric',
  'RT @EmperorBigD: You know its funny.\n\nI haven\'t seen a SINGLE person complain that they hated Last Jedi because "THEIR theory" didn\'t happe…'),
 ('Last Jedi',
  '2017-12-22 17:34:23.054049',
  'Tutvm',
  'Also I finally got to see the last Jedi today. \nStrong overall but some weird moments &amp; a terrible (though maybe ne… https://t.co/1dtfXmnjtG'),
 ('Last Jedi',
  '2017-12-22 17:34:24.041080',
  'TrisYoung28',
  'Watching The Last Jedi again by myself on Christmas day sounds great'),
 ('Last Jedi',
  '2017-12-22 17:34:25.560593',
  'TheFearlessHero',
  'RT @z0mgItsHutch: "Jesus Christ I can\'t believe you\'d rate The Last Jedi so high. Seriously what is wrong with you??"\n\nGood question, and t…'),
 ('Last Jedi',
  '2017-12-22 17:34:26.292106',
  'Devidraws',
  'The last Jedi was fucking boring as hell'),
 ('Last Jedi',
  '2017-12-22 17:34:28.066004',
  'EmmaLindhagen',
  'Without having seen the last Jedi I can hones

In [12]:
relevant_rows = c.execute("SELECT COUNT (*) FROM twitter WHERE filter=?", (filter_input,))
count_of_relevant_tweets = relevant_rows.fetchall()

In [13]:
count_of_relevant_tweets

[(51,)]

In [14]:
filtered_tweets = c.execute("SELECT tweet FROM twitter WHERE filter=?", (filter_input,))

def clean_tweets(filtered_tweets):
    clean_tweets = []
    for tweet in filtered_tweets:
        tweet1 = str(tweet)
        tweet2 = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet1).split())
        clean_tweets.append(tweet2)
    return clean_tweets

In [15]:
just_tweets = clean_tweets(filtered_tweets)

In [16]:
len(just_tweets)

51

In [17]:
def analyze_that(just_tweets):
    pos_list = []
    neg_list = []
    for tweet in just_tweets:
        tweet_tb = TextBlob(tweet, analyzer=NaiveBayesAnalyzer())
        sent = tweet_tb.sentiment
        pos_list.append(sent.p_pos)
        neg_list.append(sent.p_neg)
    
    avg_pos = sum(pos_list) / len(pos_list)
    avg_neg = sum(neg_list) / len(neg_list)
    
    return avg_pos, avg_neg # avg percentage positive & percentage negative

In [20]:
analyze_that(just_tweets)

(0.7905551464839482, 0.20944485351605158)